In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import string
from sklearn.preprocessing import OrdinalEncoder
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'


In [3]:
#customize possible missing value
missing_value_formats = ["n.a.","?","NA","n/a", "na", "--"]
train=pd.read_csv("/Users/bijiben/Desktop/BDA/advancedanalytics/assignment1/train.csv", na_values = missing_value_formats)

In [4]:
clear_list=['property_id', 'host_about', 'host_response_time', 'host_response_rate', 'host_nr_listings', 'host_nr_listings_total', 
      'host_verified', 'booking_price_covers', 'booking_min_nights', 'booking_max_nights', 'property_beds']

In [5]:
df=train[clear_list]
df.head()

,property_id,host_about,host_response_time,host_response_rate,host_nr_listings,host_nr_listings_total,host_verified,booking_price_covers,booking_min_nights,booking_max_nights,property_beds
0,1,NaN,within a few hours,100.0,1.0,1.0,"email, phone, reviews",1,1,1125,1.0
1,2,NaN,within a few hours,75.0,1.0,1.0,"email, phone, reviews",1,2,15,2.0
2,3,ik ben een expat die alleen mid weeks in Antwe...,within an hour,100.0,1.0,1.0,"email, phone, reviews",1,2,1125,1.0
3,4,"Hello, I was born in the Netherlands and i sp...",within an hour,100.0,2.0,2.0,"email, phone, facebook, reviews, jumio",1,1,1125,1.0
4,5,"Hello, I was born in the Netherlands and i sp...",within an hour,100.0,2.0,2.0,"email, phone, facebook, reviews, jumio",1,1,1125,1.0


In [6]:
#investigate data type 
types=df.dtypes
print(types)

property_id                 int64
host_about                 object
host_response_time         object
host_response_rate        float64
host_nr_listings          float64
host_nr_listings_total    float64
host_verified              object
booking_price_covers        int64
booking_min_nights          int64
booking_max_nights          int64
property_beds             float64
dtype: object


In [7]:
#check missingness in data
df.isnull().sum()
#quite lot null values in data, can not directly drop the row

property_id                  0
host_about                3107
host_response_time        1461
host_response_rate        1461
host_nr_listings             1
host_nr_listings_total       1
host_verified                0
booking_price_covers         0
booking_min_nights           0
booking_max_nights           0
property_beds               11
dtype: int64

# impute missingness and encoding

In [8]:
#host_nr_listings and host_nr_listings_total
#since the outliers in the nr-listing
#replace missingness by median
df['host_nr_listings'] = df['host_nr_listings'].fillna(df['host_nr_listings'].median())
df['host_nr_listings_total'] = df['host_nr_listings_total'].fillna(df['host_nr_listings_total'].median())



In [10]:
#11 missingness
#hence, use k nereast neighbour imputation
from sklearn.impute import KNNImputer
imputer = KNNImputer()
df['property_beds']=imputer.fit_transform(df[['property_beds']])


In [11]:
#check missingness again
df.isnull().sum()


property_id                  0
host_about                3107
host_response_time        1461
host_response_rate        1461
host_nr_listings             0
host_nr_listings_total       0
host_verified                0
booking_price_covers         0
booking_min_nights           0
booking_max_nights           0
property_beds                0
dtype: int64

In [12]:
#there are 1461 missingness in host response time and host response rate
#can not directly ignore or impute with median or mode
#create a value represent missingness
print("Null values:", df.host_response_time.isna().sum())
print(f"Proportion: {round((df.host_response_time.isna().sum()/len(df))*100, 1)}%")

df.host_response_time.fillna("unknown", inplace=True)
df.host_response_time.value_counts(normalize=True)
#5 category in total

Null values: 1461
Proportion: 22.5%


within an hour        0.379061
unknown               0.224942
within a few hours    0.206467
within a day          0.159045
a few days or more    0.030485
Name: host_response_time, dtype: float64

In [13]:
print("Mean host response rate:", round(df['host_response_rate'].mean(),0))
print("Median host response rate:", df['host_response_rate'].median())
print(f"Proportion of 100% host response rates: {round(((df.host_response_rate == 100.0).sum()/df.host_response_rate.count())*100,1)}%")

Mean host response rate: 92.0
Median host response rate: 100.0
Proportion of 100% host response rates: 70.0%


In [14]:
#encode host_response time
#unkonw is encoded as 1
ord_enc = OrdinalEncoder(categories=[['unknown', 'within an hour', 'within a few hours', 'within a day', 'a few days or more']])
df["responsetime_code"] = ord_enc.fit_transform(df[["host_response_time"]])
df[["host_response_time", "responsetime_code"]].head(20)


,host_response_time,responsetime_code
0,within a few hours,2.0
1,within a few hours,2.0
2,within an hour,1.0
3,within an hour,1.0
4,within an hour,1.0
5,within an hour,1.0
6,a few days or more,4.0
7,within an hour,1.0
8,within a few hours,2.0
9,within a day,3.0


In [15]:
#use KNN to impute host response rate
imputer = KNNImputer()
df['host_response_rate']=imputer.fit_transform(df[['host_response_rate']])


In [16]:
#process host_verified
df.host_verified.value_counts()

email, phone, reviews                                           1742
email, phone, reviews, jumio                                    1461
email, phone, facebook, reviews, jumio                           437
email, phone, facebook, reviews                                  437
email, phone                                                     298
                                                                ... 
email, phone, facebook, google, reviews, kba, work_email           1
email, phone, manual_online, manual_offline                        1
phone, facebook, jumio, offline_government_id, government_id       1
reviews, jumio                                                     1
email, phone, linkedin, reviews, work_email                        1
Name: host_verified, Length: 147, dtype: int64

In [17]:
#convert verifying document with and without official id into booleen and binary data
df['logic_host_verified']= df.host_verified.str.contains("offline_government_id|government_id|driver’s license|passport|identity card|id|visa", regex=True)
df['logic_host_verified'].head(10)


0    False
1    False
2    False
3    False
4    False
5    False
6    False
7     True
8    False
9    False
Name: logic_host_verified, dtype: bool

In [18]:
train=pd.read_csv("/Users/bijiben/Desktop/BDA/advancedanalytics/assignment1/combine_train.csv", engine='python', dtype={'user_id': int})

In [19]:
train=train[['property_id', 'property_max_guests']]

In [20]:
df=pd.merge(df, train, on='property_id')

In [21]:
#create new column indicating possible addition bed
df['extra_beds']=df['property_max_guests']-df['booking_price_covers']
df['extra_beds'].head()

0    0
1    3
2    1
3    1
4    1
Name: extra_beds, dtype: int64

In [22]:
df.isnull().sum()

property_id                  0
host_about                3107
host_response_time           0
host_response_rate           0
host_nr_listings             0
host_nr_listings_total       0
host_verified                0
booking_price_covers         0
booking_min_nights           0
booking_max_nights           0
property_beds                0
responsetime_code            0
logic_host_verified          0
property_max_guests          0
extra_beds                   0
dtype: int64

In [23]:
#drop unnecessary columns：host_about, host_verified(since already convert to binary variable)
df=df.drop(['host_about', 'host_verified', 'property_max_guests','host_response_time'], axis=1)
df.head(5)

,property_id,host_response_rate,host_nr_listings,host_nr_listings_total,booking_price_covers,booking_min_nights,booking_max_nights,property_beds,responsetime_code,logic_host_verified,extra_beds
0,1,100.0,1.0,1.0,1,1,1125,1.0,2.0,False,0
1,2,75.0,1.0,1.0,1,2,15,2.0,2.0,False,3
2,3,100.0,1.0,1.0,1,2,1125,1.0,1.0,False,1
3,4,100.0,2.0,2.0,1,1,1125,1.0,1.0,False,1
4,5,100.0,2.0,2.0,1,1,1125,1.0,1.0,False,1


In [25]:
#export clean data
df.to_csv('siwen_train.csv',index=False)
